### The "Talker vs. Doer" (Structured Tool Invocation)
This demo supports the slides on Why Tools are Needed and Combining Tools with Structured Outputs. It shows how a raw AI response is converted into a validated, executable action.

### Define the Structured Tool

In [ ]:
from pydantic import BaseModel, Field, field_validator
from datetime import datetime


# Use Pydantic to ensure the tool never receives corrupted data
class SendEmailTask(BaseModel):
    recipient: str = Field(..., description="The email address of the receiver")
    subject: str = Field(..., description="The subject line of the email")
    body: str = Field(..., description="The main content of the message")
    priority: int = Field(default=3, ge=1, le=5, description="Priority from 1-5")

    # The modern replacement for @validator
    @field_validator("recipient")
    @classmethod
    def validate_email(cls, v: str) -> str:
        if "@" not in v:
            raise ValueError("Invalid email format")
        return v

# TOOL
# The application executes the tool and logs the action
def execute_email_tool(task: SendEmailTask):
    print(f"--- [AUDIT LOG: {datetime.now()}] ---")
    print(f"ACTION: Sending Email to {task.recipient}")
    print(f"SUBJECT: {task.subject}")
    print(f"PRIORITY: {task.priority}")
    return {"status": "success", "message_id": "msg_98765"}

### Simulating the AI's Decision

In [2]:
# The Model analyzes the request and passes structured arguments
raw_ai_decision = {
    "recipient": "hr@enterprise.com",
    "subject": "Q1 Performance Review",
    "body": "Please find the attached report for the first quarter.",
    "priority": 2,
}

print("AI decides to use the 'SendEmail' tool...")
try:
    # Validation step 
    validated_task = SendEmailTask(**raw_ai_decision)
    result = execute_email_tool(validated_task)
    print(f"\nResult: {result['status']}")
except Exception as e:
    print(f"Safety Blocked Execution: {e}")

AI decides to use the 'SendEmail' tool...
--- [AUDIT LOG: 2026-02-25 08:18:10.791946] ---
ACTION: Sending Email to hr@enterprise.com
SUBJECT: Q1 Performance Review
PRIORITY: 2

Result: success


### Multi-Step Chained Reasoning

In [3]:
# A toolkit organizes related tools to reduce cognitive load
class InventoryToolkit:
    def __init__(self):
        self.db = {"laptop": 5, "monitor": 12, "keyboard": 0}
    
    def check_stock(self, item: str):
        # Returns information to feed the next step
        print(f"Checking stock for: {item}...")
        return self.db.get(item.lower(), 0)

    def trigger_restock(self, item: str, quantity: int):
        # AI executes workflows rather than single actions
        print(f"Restocking {quantity} units of {item}...")
        self.db[item.lower()] += quantity
        return f"New balance: {self.db[item.lower()]}"

toolkit = InventoryToolkit()

### Executing the Workflow

In [4]:
# The model plans a sequence of actions
def run_autonomous_agent(query):
    print(f"User Query: {query}\n")

    # Step 1: Reason and Check 
    item = "keyboard"
    current_stock = toolkit.check_stock(item)

    # Step 2: Decision Step 
    if current_stock == 0:
        print(f"Reasoning: {item} is out of stock. Initiating restock...")
        # Step 3: Take Action 
        update_msg = toolkit.trigger_restock(item, 10)
        print(f"Final Output: {update_msg}")
    else:
        print(f"Reasoning: Stock is sufficient ({current_stock}). No action needed.")


run_autonomous_agent("Ensure we have enough keyboards for the new hires.")

User Query: Ensure we have enough keyboards for the new hires.

Checking stock for: keyboard...
Reasoning: keyboard is out of stock. Initiating restock...
Restocking 10 units of keyboard...
Final Output: New balance: 10
